In [163]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchmetrics.classification import MulticlassF1Score

from transformers import ElectraModel, ElectraTokenizer

from sklearn.model_selection import train_test_split

import os
import random
import time
import pandas as pd

# Data Preparation

## Loading

In [123]:
file_list = os.listdir('./dataset')
file_path = []
for file_name in file_list:
    tmp_path = os.path.join('./dataset', file_name)
    file_path.append(tmp_path)
print(file_path)
file_path.sort()
print(file_path)

# './dataset/ulsan_2021-2019_NER_v1.xlsx' start_num 1
# './dataset/port_NER_v1.xlsx' start_num 1
# './dataset/eshc_NER_v1.xlsx' start_num 0
# './dataset/port_NER_v1.xlsx' == './dataset/ulsan_2022_NER_v1.xlsx' (동일한 데이터 )


['./dataset/ulsan_2021-2019_NER_v1.xlsx', './dataset/port_NER_v1.xlsx', './dataset/ulsan_2022_NER_v1.xlsx', './dataset/eshc_NER_v1.xlsx']
['./dataset/eshc_NER_v1.xlsx', './dataset/port_NER_v1.xlsx', './dataset/ulsan_2021-2019_NER_v1.xlsx', './dataset/ulsan_2022_NER_v1.xlsx']


In [124]:
load_df = lambda x: pd.read_excel(x, names=['src', 'tar'])
dfs = [load_df(x) for x in file_path]
df_4, df_2, df_1, df_3 = dfs
df_1, df_2, df_3, df_4 = df_1.reset_index(), df_2.reset_index(), df_3.reset_index(), df_4.reset_index()

# train = pd.read_excel("./ulsan_2021-2019_NER_v1.xlsx", names=['src', 'tar'])
# train = train.reset_index()
# for i, row in train.iterrows():
#     print(row['index'], row.src, row.tar)
#     # print(row)

# (df_2 == df_3).nunique()

# df_1, df_2, df_4

## Cleansing

In [125]:
def preprocess(df):
    df['src'] = df['src'].str.replace("．", ".", regex=False)
    df['tar'] = df['tar'].str.replace("NUMB", "NUM_B", regex=False)
    df['tar'] = df['tar'].str.replace("TRM_b", "TRM_B", regex=False)
    df['tar'] = df['tar'].str.replace("WRK_b", "WRK_B", regex=False)
    df['tar'] = df['tar'].str.replace("NUM_b", "NUM_B", regex=False)
    df['tar'] = df['tar'].str.replace("AFW_B", "AWF_B", regex=False)
    df['tar'] = df['tar'].str.replace("ANM_b", "ANM_B", regex=False)
    df['tar'] = df['tar'].str.replace("NIM_B", "NUM_B", regex=False)
    df['tar'] = df['tar'].str.replace("LOC_b", "LOC_B", regex=False)
    df['tar'] = df['tar'].str.replace("AFW_b", "AFW_B", regex=False)
    df['tar'] = df['tar'].str.replace("(삭제)", "-", regex=False)
    df['tar'] = df['tar'].str.replace("TIM_b", "TIM_B", regex=False)
    df = df.fillna("-")
    df['src'] = df['src'].astype(str)
    df['tar'] = df['tar'].astype(str)
    df['src'] = df['src'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', "", regex=True)
    return df

In [126]:
df_1 = preprocess(df_1)
df_2 = preprocess(df_2)
df_4 = preprocess(df_4)

In [127]:
# df_1[:50] # start_num 1
# df_2[:50] # start_num 1
# df_4[:50] # start_num 0

In [128]:
# df_1.isnull().any(), df_2.isnull().any(), df_4.isnull().any() # not found

## Reformat / Tokenization / Integrity Check

In [129]:
train = pd.concat([df_1, df_2, df_4])

In [130]:
label = train['tar'].unique().tolist()
label_dict = {word:i for i, word in enumerate(label)}
label_dict.update({"[PAD]":len(label_dict)})
index_to_ner = {i:j for j, i in label_dict.items()}

tag_list = list(label_dict.keys())
tag_list.sort()

print(label_dict), print(), print(index_to_ner), print(), print(tag_list)

{'-': 0, 'DAT_B': 1, 'TIM_I': 2, 'LOC_B': 3, 'LOC_I': 4, 'NUM_B': 5, 'TIM_B': 6, 'CVL_B': 7, 'ANM_B': 8, 'MAT_B': 9, 'MAT_I': 10, 'WRK_B': 11, 'WRK_I': 12, 'AWF_B': 13, 'AFW_I': 14, 'AFW_O': 15, 'TRM_B': 16, 'CVL_I': 17, 'DAT_I': 18, 'MAT)B': 19, 'N': 20, 'MAT+I': 21, 'WRK_': 22, 'ANM_I': 23, 'AFW)B': 24, ',MAT_B': 25, 'TIOM_I': 26, 'LOC_': 27, 'AFW0B': 28, 'DAT_II': 29, 'WRL_I': 30, 'AFW_B': 31, 'DAT+I': 32, 'TRM_I': 33, 'DAT0B': 34, 'AFW)I': 35, 'AFW _B': 36, 'NUM_I': 37, 'AF': 38, '[PAD]': 39}

{0: '-', 1: 'DAT_B', 2: 'TIM_I', 3: 'LOC_B', 4: 'LOC_I', 5: 'NUM_B', 6: 'TIM_B', 7: 'CVL_B', 8: 'ANM_B', 9: 'MAT_B', 10: 'MAT_I', 11: 'WRK_B', 12: 'WRK_I', 13: 'AWF_B', 14: 'AFW_I', 15: 'AFW_O', 16: 'TRM_B', 17: 'CVL_I', 18: 'DAT_I', 19: 'MAT)B', 20: 'N', 21: 'MAT+I', 22: 'WRK_', 23: 'ANM_I', 24: 'AFW)B', 25: ',MAT_B', 26: 'TIOM_I', 27: 'LOC_', 28: 'AFW0B', 29: 'DAT_II', 30: 'WRL_I', 31: 'AFW_B', 32: 'DAT+I', 33: 'TRM_I', 34: 'DAT0B', 35: 'AFW)I', 36: 'AFW _B', 37: 'NUM_I', 38: 'AF', 39: '[PA

(None, None, None, None, None)

In [131]:
'-'
'AFW_B'
'AWF_B'
'AFW _B'
'AFW)B'
'AFW0B'
'AFW)I'
'AFW_I'
'AFW_O'
'AF'
'ANM_B'
'ANM_I'
'CVL_B'
'CVL_I'
'DAT_B'
'DAT_I'
'DAT+I'
'DAT0B'
'DAT_II'
'LOC_B'
'LOC_'
'LOC_I'
'MAT_B'
',MAT_B'
'MAT)B'
'MAT+I'
'MAT_I'
'NUM_B'
'NUM_I'
'N'
'TIM_B'
'TIM_I'
'TIOM_I'
'TRM_B'
'TRM_I'
'WRK_B'
'WRK_I'
'WRL_I'
'WRK_'
'[PAD]'

'[PAD]'

In [132]:
df_1_list = [list(x) for x in df_1[['index', 'src', 'tar']].to_numpy()] # start_num 1
df_2_list = [list(x) for x in df_2[['index', 'src', 'tar']].to_numpy()] # start_num 1
df_4_list = [list(x) for x in df_4[['index', 'src', 'tar']].to_numpy()] # start_num 0

In [133]:
def makeTups(data, start_num):
    tups = []
    temp_tup = []
    temp_tup.append(data[0][1:])
    for i, j, k in data: # index src tgt
        if i != start_num:
            temp_tup.append([j,k])
        if i == start_num:
            if len(temp_tup) != 0:
                tups.append(temp_tup)
                temp_tup = []
                temp_tup.append([j,k])

    tups[0].pop(0)
    return tups

In [134]:
tups_1 = makeTups(df_1_list, 1)
tups_2 = makeTups(df_2_list, 1)
tups_4 = makeTups(df_4_list, 0)
print(len(tups_1), len(tups_2), len(tups_4))

2236 1299 1028


In [135]:
tups = tups_1 + tups_2 + tups_4
print(len(tups))

4563


### Check If Empty

In [136]:
# check and remove if there is any empty row

empties = []
for i, row in enumerate(tups):
    if len(row) < 2 and row[0][0] == '':
        # print(i, row)
        empties.append(i)

empties.sort(reverse=True)

print(f'{len(empties)} empty elements')

for idx in empties:
    tups.pop(idx)

for i, row in enumerate(tups):
    if len(row) < 2 and row[0][0] == '':
        print(i, row)

len(tups)

16 empty elements


4547

### Load Tokenizer and Check Special Tokens

In [137]:
tokenizer = ElectraTokenizer.from_pretrained('./tokenizer/')

In [138]:
init_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
init_token, pad_token, unk_token, sep_token

('[CLS]', '[PAD]', '[UNK]', '[SEP]')

In [139]:
sentences = []
targets = []
for tup in tups:
    sentence = []
    target = []
    # sentence.append(init_token)
    # target.append(label_dict['-'])
    for i, j in tup:
        sentence.append(i)
        target.append(j)
    # sentence.append(sep_token)
    # target.append(label_dict['-'])
    sentences.append(sentence)
    targets.append(target)

In [140]:
# k = random.randrange(len(sentences))
# print(f'random inddex k is {k}')
# print(len(sentences)), print(len(targets))
# print(len(sentences[k])), print(len(targets[k]))
# print(sentences[k]), print(targets[k])

### Tokenization and Label Matching

In [141]:
'-'
'AFW_B'
'AWF_B'
'AFW _B'
'AFW)B'
'AFW0B'
'AFW)I'
'AFW_I'
'AFW_O'
'AF'
'ANM_B'
'ANM_I'
'CVL_B'
'CVL_I'
'DAT_B'
'DAT_I'
'DAT+I'
'DAT0B'
'DAT_II'
'LOC_B'
'LOC_'
'LOC_I'
'MAT_B'
',MAT_B'
'MAT)B'
'MAT+I'
'MAT_I'
'NUM_B'
'NUM_I'
'N'
'TIM_B'
'TIM_I'
'TIOM_I'
'TRM_B'
'TRM_I'
'WRK_B'
'WRK_I'
'WRL_I'
'WRK_'
'[PAD]'

'[PAD]'

In [142]:
# tokenized_sentence_ = []
# labels_ = []

# word_ = '우리나라만세만세만세'
# label_ = 'WRK_B'

# tokenized_word_ = tokenizer.tokenize(word_)
# n_subwords = len(tokenized_word_)

# tokenized_sentence_.extend(tokenized_word_)
# if label_[-1] == 'B':
#     tail = list(label_)
#     tail[-1] = 'I'
#     tail = ''.join(tail)
#     labels_.extend([label_] + [tail]*(n_subwords-1))
# else:
#     labels_.extend([label_] * n_subwords)

# tokenized_sentence_, labels_

In [143]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []
    for word, label in zip(sentence, text_labels):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        tokenized_sentence.extend(tokenized_word)

        if label[-1] == 'B':
            tail = list(label)
            tail[-1] = 'I'
            tail = ''.join(tail)
            labels.extend([label] + [tail]*(n_subwords-1))
        else:
            labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [144]:
tokenized_texts_and_labels = [
                              tokenize_and_preserve_labels(sent, labs)
                              for sent, labs in zip(sentences, targets)]

In [145]:
# # [(문장, 개체들), (문장, 개체들),...] 형식으로 저장되어 있음.
# k = random.randrange(len(tokenized_texts_and_labels))
# print(f'random inddex k is {k}')
# print(len(tokenized_texts_and_labels)), print(len(tokenized_texts_and_labels))
# print(len(tokenized_texts_and_labels[k][0])), print(len(tokenized_texts_and_labels[k][1]))
# print(tokenized_texts_and_labels[k][0]), print(), print(tokenized_texts_and_labels[k][1])

In [146]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [147]:
k = random.randrange(len(tokenized_texts))
print(f'random inddex k is {k}')
print(len(tokenized_texts)), print(len(labels))
print(len(tokenized_texts[k])), print(len(labels[k])), print()
# print(tokenized_texts[k]), print(labels[k])
for token, label in zip(tokenized_texts[k], labels[k]):
    print(token, '\t',label)

random inddex k is 4498
4547
4547
18
18

서울특별 	 LOC_B
시 	 LOC_I
소재 	 LOC_I
공사현장 	 LOC_I
내 	 LOC_I
##에 	 LOC_I
##서 	 LOC_I
골재 	 -
운반용 	 -
컨베이어 	 TRM_B
벨트 	 TRM_I
와 	 TRM_I
롤러 	 TRM_I
사이 	 -
##에 	 -
끼 	 -
##임 	 -
. 	 -


In [148]:
lengths = []
for tokens in tokenized_texts:
    lengths.append(len(tokens))

f'the longest is {max(lengths)}'

'the longest is 241'

In [149]:
tokenized_texts_list, labels_list = tokenized_texts, labels

### Add Special Tokens and Pad Inputs

In [150]:
# print(len(label_dict)), print(len(index_to_ner))
# print(label_dict)
# print(index_to_ner)
print(label_dict.keys())

dict_keys(['-', 'DAT_B', 'TIM_I', 'LOC_B', 'LOC_I', 'NUM_B', 'TIM_B', 'CVL_B', 'ANM_B', 'MAT_B', 'MAT_I', 'WRK_B', 'WRK_I', 'AWF_B', 'AFW_I', 'AFW_O', 'TRM_B', 'CVL_I', 'DAT_I', 'MAT)B', 'N', 'MAT+I', 'WRK_', 'ANM_I', 'AFW)B', ',MAT_B', 'TIOM_I', 'LOC_', 'AFW0B', 'DAT_II', 'WRL_I', 'AFW_B', 'DAT+I', 'TRM_I', 'DAT0B', 'AFW)I', 'AFW _B', 'NUM_I', 'AF', '[PAD]'])


In [151]:
def make_inputs(tokenized_texts_list, labels_list, max_length):
    max_length = max_length
    input_tokens = []
    input_mask = []
    input_labels = []
    num = len(tokenized_texts_list)
    for i in range(num):
        data = tokenized_texts_list[i]
        labels = labels_list[i]
        tmp_tokens = [init_token] + data
        tmp_labels = ['-'] + labels
        if len(tmp_tokens) < max_length-1:
            tmp_tokens += [sep_token]
            tmp_labels += ['-']
            tmp_mask = [1] * len(tmp_tokens)
            for _ in range(max_length-len(tmp_tokens)):
                tmp_tokens.append(pad_token)
                tmp_labels.append('[PAD]')
                tmp_mask.append(0)
            input_tokens.append(tmp_tokens)
            input_labels.append(tmp_labels)
            input_mask.append(tmp_mask)
        elif len(tmp_tokens) > max_length-1:
            tmp_tokens = tmp_tokens[:max_length-1] + [sep_token]
            tmp_labels = tmp_labels[:max_length-1] + ['-']
            tmp_mask = [1] * max_length
            input_tokens.append(tmp_tokens)
            input_labels.append(tmp_labels)
            input_mask.append(tmp_mask)
        elif len(tmp_tokens) == max_length-1:
            tmp_tokens += [sep_token]
            tmp_labels += [label_dict['-']]
            tmp_mask = [1] * max_length
            input_tokens.append(tmp_tokens)
            input_labels.append(tmp_labels)
            input_mask.append(tmp_mask)

    return input_tokens, input_mask, input_labels

In [152]:
max_length = 256
input_tokens, input_mask, input_labels = make_inputs(tokenized_texts, labels, max_length)

In [153]:
print(f'length of input_tokens: {len(input_tokens)}')
print(f'length of input_labels: {len(input_labels)}')
print(f'length of input_mask: {len(input_mask)}')

k = random.randrange(len(input_tokens))
print(f'random inddex k is {k}')
print(len(input_tokens[k])), print(len(input_labels[k])), print(len(input_mask[k]))
print(), print(input_tokens[k]), print(), print(input_labels[k]), print(), print(input_mask[k])

length of input_tokens: 4547
length of input_labels: 4547
length of input_mask: 4547
random inddex k is 2816
256
256
256

['[CLS]', '경기도', '여주', '시', '가남읍', '내', 'OO', '목장', '사업장', '##에', '##서', '노후화', '된', '축사', '지붕', '교체', '작업', '중', '재해자', '가', '선라이트', '를', '밟', '##아', '파손', '##되', '##면', '##서', '떨어져', '사망', '##함', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

(None, None, None, None, None, None)

### Check Lengths

In [154]:
num = len(input_tokens)
for i in range(num):
    curr_tokens = input_tokens[i]
    curr_label = input_labels[i]
    curr_mask = input_mask[i]
    
    count_tokens, count_label, count_mask = 0,0,0
    for count, token in enumerate(curr_tokens):
        if token == '[PAD]':
            break
    count_tokens = count
    for count, label in enumerate(curr_label):
        if label == 39:
            break
    count_label = count
    for count, mask in enumerate(curr_mask):
        if mask == 0:
            break
    count_mask = count
    if not (count_tokens == count_label == count_mask):
        print(count_tokens, count_label, count_mask)

76 255 76
119 255 119
28 255 28
68 255 68
49 255 49
32 255 32
120 255 120
77 255 77
53 255 53
72 255 72
57 255 57
51 255 51
52 255 52
70 255 70
36 255 36
42 255 42
114 255 114
75 255 75
48 255 48
47 255 47
99 255 99
50 255 50
113 255 113
43 255 43
91 255 91
142 255 142
51 255 51
117 255 117
56 255 56
49 255 49
65 255 65
40 255 40
40 255 40
60 255 60
58 255 58
63 255 63
77 255 77
81 255 81
66 255 66
67 255 67
50 255 50
54 255 54
74 255 74
45 255 45
92 255 92
64 255 64
52 255 52
67 255 67
78 255 78
52 255 52
67 255 67
99 255 99
66 255 66
98 255 98
109 255 109
74 255 74
58 255 58
57 255 57
69 255 69
76 255 76
47 255 47
96 255 96
32 255 32
71 255 71
86 255 86
55 255 55
86 255 86
80 255 80
57 255 57
65 255 65
59 255 59
59 255 59
85 255 85
63 255 63
82 255 82
69 255 69
52 255 52
88 255 88
93 255 93
88 255 88
148 255 148
70 255 70
103 255 103
56 255 56
54 255 54
62 255 62
40 255 40
103 255 103
65 255 65
93 255 93
61 255 61
96 255 96
51 255 51
58 255 58
102 255 102
52 255 52
93 255 93
53 255 5

### Pair Inputs and Split to Ratio

In [155]:
input_pair = []
num = len(input_tokens)
for i in range(num):
    tmp_token_idx = input_tokens[i]
    tmp_mask = input_mask[i]
    tmp_label_idx = input_labels[i]
    input_pair.append((tmp_token_idx, tmp_mask, tmp_label_idx))

In [156]:
# k = random.randrange(len(input_tokens))
# print(input_pair[k])

데이터셋을 8:2 비율로 훈련 데이터셋과 검증 데이터셋으로 분할합니다. 데이터셋의 양이 적어 테스트셋은 생성하지 않았습니다.

In [157]:
train, valid = train_test_split(input_pair, test_size=0.2)

In [158]:
print(len(train)), print(len(valid))
# print(train[0]), print(valid[0])

3637
910


(None, None)

In [159]:
# temp = []
# for e in train[0][0]:
#     encoded = tokenizer.convert_tokens_to_ids(str(e))
#     temp.append(encoded)
# print(temp)

### Delcare Dataset (Iterator)

pytorch에서 데이터를 받아들일 수 있도록 커스텀 데이터셋을 생성합니다. 이 때 문장을 구성하는 토큰들을 고유한 인덱스 번호로 변환합니다. 마찬가지로 레이블도 인덱스 번호로 변환합니다.

In [160]:
class CustomDataset(Dataset): 
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    def __len__(self): 
        return len(self.data)

    def __getitem__(self, idx):
        tokens = self.data[idx][0]
        masks = self.data[idx][1]
        labels = self.data[idx][2]
        token_ids = []
        for token in tokens:
            ids = tokenizer.convert_tokens_to_ids(str(token))
            token_ids.append(ids)
        label_ids = []
        for label in labels:
            label_ids.append(label_dict[label])
                
        return (torch.LongTensor(token_ids), torch.LongTensor(masks), torch.LongTensor(label_ids))

In [161]:
trainset = CustomDataset(train, tokenizer)
validset = CustomDataset(valid, tokenizer)

배치 크기가 128인 데이터로더를 생성합니다.

In [162]:
train_loader = DataLoader(trainset, batch_size = 128, shuffle = True)
valid_loader = DataLoader(validset, batch_size = 128, shuffle = True)

In [39]:
for i, el in enumerate(train_loader):
    print(len(el[0][0]))
    print(len(el[1][0]))
    print(len(el[2][0]))
    # print(el[0][0])
    # print(el[1][0])
    # print(el[2][0])
    break

256
256
256


# Create Model / Training / Evaluation / Inference

모델을 생성합니다.

In [40]:
class BERTPoSTagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, mask):
        
        #text = [batch size, sent len]
        embedded = self.dropout(self.bert(text, mask)[0])
        #embedded = [batch size, seq len, emb dim]
        
        predictions = self.fc(self.dropout(embedded))
        
        
        return predictions

In [41]:
bert = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
print(len(tokenizer))

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


36223


In [42]:
OUTPUT_DIM = 40
DROPOUT = 0.25

model = BERTPoSTagger(bert,
                      OUTPUT_DIM, 
                      DROPOUT)
model.bert.resize_token_embeddings(len(tokenizer))

Embedding(36223, 768)

In [43]:
model.bert.config.to_dict()['hidden_size']

768

In [44]:
bert_result = model.bert(trainset[0][0].unsqueeze(0), trainset[0][1].unsqueeze(0))
bert_result.last_hidden_state.shape

torch.Size([1, 256, 768])

In [45]:
tagger_result = model(trainset[0][0].unsqueeze(0), trainset[0][1].unsqueeze(0))
tagger_result.shape

torch.Size([1, 256, 40])

모델을 학습시키기 위한 옵티마이저와 목적함수(cross entropy)를 호출합니다.

In [46]:
LEARNING_RATE = 5e-5

optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [47]:
criterion = nn.CrossEntropyLoss(ignore_index = 0)

GPU 연산을 위해 cuda를 호출합니다. 이 때 복수의 연산장치(GPU)가 있을 경우 모든 연산장치를 활용하여 연산의 속도를 높이도록 설정합니다.

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NGPU = torch.cuda.device_count()
if NGPU > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(NGPU)))
    # torch.multiprocessing.set_start_method('spawn', force=True)
model = model.to(device)
criterion = criterion.to(device)

정확도를 계산하기 위한 함수를 생성합니다.

In [49]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

F1 Macro and F1 Micro

In [ ]:
f1_macro = MulticlassF1Score(num_classes=40, average='macro')
f1_micro = MulticlassF1Score(num_classes=40, average='micro')

모델을 훈련시키기 위한 함수입니다.

In [50]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch[0].to(device)
        mask = batch[1].to(device)
        tags = batch[2].to(device)

        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        # print(text)
        # print()
        # print(mask)
        # print()
        # print(tags)
                
        predictions = model(text, mask)

        # print()
        # print(predictions)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        # torch.Size([32, 256, 40])
        predictions = predictions.view(-1, predictions.shape[-1]) # shape(batchsize, length, ouputdim) --> shape(batchsize, length*outputdim)
        # torch.Size([8192, 40])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

모델의 성능을 평가하기 위한 함수입니다.

In [51]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch[0].to(device)
            mask = batch[1].to(device)
            tags = batch[2].to(device)
            
            predictions = model(text, mask)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            f1_macro(predictions, tags, ignore_index=tag_pad_idx)
            f1_micro(predictions, tags, ignore_index=tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

에폭마다 걸리는 시간을 계산하기 위한 함수입니다.

In [52]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

실질적으로 모델을 학습시킵니다. 에폭마다 검증데이터셋에서 성능을 평가하여 검증데이터셋에서 가장 좋은 성능을 보여줬을 때의 모델의 weight를 저장합니다.

In [216]:
N_EPOCHS = 15

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, 0)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion, 0)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

: 

: 

학습된 모델을 이용하여 예시 문장으로 inference 합니다.

In [ ]:
model.load_state_dict(torch.load("tut2-model.pt"))
model.eval()
with torch.no_grad():
    sample = '2013년 11월 18일 인천항 4부두에서 컨테이너 작업 중 핸들러 집게가 풀리면서 피해자 끼임. 허리에 경상을 입음'
    words = tokenizer.tokenize(sample)
    tokens = tokenizer(sample)["input_ids"]
    mask = tokenizer(sample)['attention_mask']
    tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)
    tensor_mask = torch.LongTensor(mask).unsqueeze(0).to(device)
    logit = model(tensor, tensor_mask)
    pred = torch.argmax(logit, dim=-1).squeeze().detach().cpu().tolist()
    for i in range(len(words)):
        print(words[i], index_to_ner[pred[i+1]])

2013 O
##년 O
11 O
##월 O
18 O
##일 O
인천 O
항 O
4 O
부두 O
에서 O
컨테이너 C
작업 C
중 O
핸들러 CE
집게 CE
##가 CE
풀리 CE
##면 CE
##서 CE
피해자 O
끼 CE
##임 CE
. O
허리 O
##에 O
경상 E
##을 O
입 O
##음 O


검증 데이터셋에서 5%의 데이터를 임의로 추출하여 inference 한 다음 그 결과를 csv파일로 저장합니다.

In [ ]:
import random
import numpy as np

num = len(valid)
random_choice = np.random.choice(num, int(num*0.05), replace=False).tolist()
selected_sample = []

for i in random_choice:
    selected_sample.append(valid[i])
    
print(len(selected_sample))

38


In [ ]:
model.eval()
with torch.no_grad():
    sample_results = []
    for idx, _ in enumerate(selected_sample):
        item = selected_sample[idx]
        token_ids = list(map(lambda x:tokenizer.convert_tokens_to_ids(x), item[0]))
        input_tensor = torch.LongTensor(token_ids).unsqueeze(0).to(device)
        mask_tensor = torch.LongTensor(item[1]).unsqueeze(0).to(device)
        output = model(input_tensor, mask_tensor)
        pred = torch.argmax(output, dim=-1).squeeze().detach().cpu().tolist()
        sample_results.append(pred)

In [ ]:
f = open('sample_result.csv', 'w', encoding='utf-8-sig')
idx_to_label = {0:"pad", 1:"O", 2:"C", 3:"CE", 4:"E"}
for idx, item in enumerate(selected_sample):
    all_tokens = item[0]
    if '[PAD]' not in all_tokens:
        continue
    pad_loc = all_tokens.index('[PAD]')
    tokens = all_tokens[:pad_loc][1:-1]
    labels = item[-1][:pad_loc][1:-1]
    predicts = sample_results[idx][:pad_loc][1:-1]
    length = len(tokens)
    for i in range(length):
        token = tokens[i]
        label = labels[i]
        predict = idx_to_label[predicts[i]]
        f.write(token+','+label+','+predict+'\n')
    f.write('\n')
f.close()

In [ ]:
df = pd.read_csv('./8-9_cause_effec.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './8-9_cause_effec.csv'

In [ ]:
num = len(df)
idx_to_label = {0:"pad", 1:"O", 2:"C", 3:"CE", 4:"E"}
result = []
for i in range(num):
    sentence = df.iloc[i,0]
    tokenizer_result = tokenizer.encode_plus(sentence)
    tokens = tokenizer.tokenize(sentence)
    input_ids = tokenizer_result["input_ids"]
    mask = tokenizer_result["attention_mask"]
    input_tensor = torch.LongTensor(input_ids).unsqueeze(0).to(device)
    mask_tensor = torch.LongTensor(mask).unsqueeze(0).to(device)
    output = model(input_tensor, mask_tensor)
    pred = torch.argmax(output, dim=-1).squeeze().detach().cpu().tolist()
    pred_label = list(map(lambda x:idx_to_label[x], pred))
    result.append([tokens, pred_label[1:-1]])
    

In [ ]:
f = open('./8-9_cause_effec_result.csv', 'w', encoding='utf-8-sig')
for item in result:
    length = len(item[0])
    for i in range(length):
        f.write(item[0][i]+','+item[1][i]+'\n')
    f.write('\n')
f.close()